###  Roboflow Datasets

A continuación se encuentran los enlaces a los conjuntos de datos utilizados en este proyecto:

- 🔹 **[Tile Camera Fruits Dataset](https://app.roboflow.com/university-xib5d/tile-camera-fruits-ojojf/1)**  
  Conjunto de datos basado en imágenes por mosaicos (tiles) de frutas capturadas por cámara.

- 🔹 **[Normal Fruits Dataset](https://universe.roboflow.com/isdp-n2uvz/fyp-molyt)**  
  Conjunto de datos estándar de frutas, sin segmentación por mosaicos.

In [1]:
%cd /content/drive/MyDrive/VisionIIA
%pwd

/content/drive/MyDrive/VisionIIA


'/content/drive/MyDrive/VisionIIA'

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.5 MB/s eta 0:00:00


In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 104.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR-API-KEY-ROBOFLOW")
project = rf.workspace("isdp-n2uvz").project("fyp-molyt")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to FYP-1 in yolov11:: 100%|██████████| 224/224 [00:01<00:00, 161.40it/s]


In [ ]:

from ultralytics import YOLO
import os
from PIL import Image
import numpy as np

# ============================================
# 1. ANALIZAR TAMAÑOS DE IMÁGENES
# ============================================
image_folder = '/content/drive/MyDrive/VisionIIA/FYP-1/train/images'

# Obtener lista de imágenes
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

if not image_files:
    print("⚠️ No se encontraron imágenes en la carpeta")
else:
    widths = []
    heights = []

    print(f"📊 Analizando {len(image_files)} imágenes...\n")

    # Analizar primeras 100 imágenes (o todas si son menos)
    sample_size = min(100, len(image_files))

    for img_file in image_files[:sample_size]:
        img_path = os.path.join(image_folder, img_file)
        try:
            with Image.open(img_path) as img:
                widths.append(img.width)
                heights.append(img.height)
        except Exception as e:
            print(f"Error con {img_file}: {e}")

    if widths and heights:
        # Estadísticas
        avg_width = np.mean(widths)
        avg_height = np.mean(heights)
        max_width = np.max(widths)
        max_height = np.max(heights)
        min_width = np.min(widths)
        min_height = np.min(heights)

        print("=" * 50)
        print("📏 ESTADÍSTICAS DE TAMAÑO DE IMÁGENES")
        print("=" * 50)
        print(f"Ancho promedio:  {avg_width:.0f}px")
        print(f"Alto promedio:   {avg_height:.0f}px")
        print(f"Ancho máximo:    {max_width:.0f}px")
        print(f"Alto máximo:     {max_height:.0f}px")
        print(f"Ancho mínimo:    {min_width:.0f}px")
        print(f"Alto mínimo:     {min_height:.0f}px")
        print("=" * 50)

        # Recomendar tamaño imgsz
        max_dim = max(max_width, max_height)

        # Tamaños comunes para YOLO (múltiplos de 32)
        yolo_sizes = [320, 416, 480, 512, 640, 800, 960, 1024, 1280]

        recommended_size = 640  # Default
        for size in yolo_sizes:
            if size >= max_dim * 0.8:  # Al menos 80% del tamaño máximo
                recommended_size = size
                break

        print(f"\n💡 TAMAÑO RECOMENDADO: imgsz={recommended_size}")
        print(f"   - Para mayor velocidad: imgsz=640")
        print(f"   - Para mayor precisión: imgsz=800 o imgsz=1024")
        print("=" * 50)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
📊 Analizando 95 imágenes...

📏 ESTADÍSTICAS DE TAMAÑO DE IMÁGENES
Ancho promedio:  640px
Alto promedio:   640px
Ancho máximo:    640px
Alto máximo:     640px
Ancho mínimo:    640px
Alto mínimo:     640px

💡 TAMAÑO RECOMENDADO: imgsz=512
   - Para mayor velocidad: imgsz=640
   - Para mayor precisión: imgsz=800 o imgsz=1024


In [ ]:
from ultralytics import YOLO

# Definir la ruta al archivo data.yaml
data_yaml = '/content/drive/MyDrive/VisionIIA/FYP-1/data.yaml'

# Inicializar el modelo YOLOv11 pre-entrenado en COCO para segmentación
model = YOLO('https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n-seg.pt')

# Entrenar el modelo (fine-tuning)
results = model.train(
    data=data_yaml,
    epochs=100,  # Número de épocas
    imgsz=640,   # Tamaño de la imagen de entrada
    batch=16,    # Tamaño del batch
    name='yolov11n_seg_custom_transfer',  # Nombre para guardar los resultados
    pretrained=True,  # Usar pesos pre-entrenados
    freeze=10,  # Congelar las primeras 10 capas
    lr0=0.001,  # Tasa de aprendizaje inicial
    #patience=50,  # Paciencia para early stopping
    save=True,  # Guardar los mejores checkpoints
    device='0'  # Usar GPU 0 (cambia a 'cpu' si no tienes GPU)
)

# Evaluar el modelo en el conjunto de validación
val_results = model.val()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Found https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n-seg.pt locally at weights/yolo11n-seg.pt
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/VisionIIA/FYP-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.

In [ ]:
from ultralytics import YOLO
import glob
import cv2
import os
from google.colab.patches import cv2_imshow
from IPython.display import display, Image
import numpy as np

# Cargar el modelo entrenado
model = YOLO('/content/drive/MyDrive/VisionIIA/runs/segment/yolov11n_seg_custom_transfer/weights/best.pt')  # Ajusta esta ruta a donde se guardó tu mejor modelo

# Directorio que contiene las imágenes para validar
image_dir = '/content/drive/MyDrive/VisionIIA/FYP-1/test/images'  # Ajusta esta ruta a tu directorio de imágenes

# Obtener todas las imágenes en el directorio
image_files = glob.glob(os.path.join(image_dir, '*.jpg')) + glob.glob(os.path.join(image_dir, '*.png'))

# Crear un directorio para guardar los resultados
results_dir = 'validation_results'
os.makedirs(results_dir, exist_ok=True)

# Procesar cada imagen
for img_path in image_files:
    # Realizar la predicción
    results = model(img_path)

    # Obtener la imagen con las detecciones dibujadas
    img = results[0].plot()

    # Convertir de BGR a RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Guardar la imagen con las detecciones
    output_path = os.path.join(results_dir, os.path.basename(img_path))
    cv2.imwrite(output_path, cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))

    # Mostrar la imagen usando cv2_imshow
    cv2_imshow(img_rgb)

    # Mostrar información sobre las detecciones
    print(f"Detecciones para {os.path.basename(img_path)}:")
    for r in results:
        print(f"Número de objetos detectados: {len(r.boxes)}")
        for box in r.boxes:
            print(f"  Clase: {model.names[int(box.cls)]}, Confianza: {box.conf.item():.2f}")

    print("\n")  # Agregar una línea en blanco entre imágenes

print(f"Resultados guardados en {results_dir}")

Output hidden; open in https://colab.research.google.com to view.

In [3]:
from ultralytics import YOLO
import cv2
import os
from google.colab.patches import cv2_imshow
from IPython.display import display, HTML
import numpy as np

# Cargar el modelo entrenado
model = YOLO('/content/drive/MyDrive/VisionIIA/runs/segment/yolov11n_seg_custom_transfer/weights/best.pt')

# Ruta del video de entrada
video_path = '/content/drive/MyDrive/VisionIIA/apple.mp4'  # Ajusta esta ruta a tu video

# Crear directorio para guardar resultados
results_dir = 'validation_results_video'
os.makedirs(results_dir, exist_ok=True)

# Ruta del video de salida
output_video_path = os.path.join(results_dir, 'output_video_apple_NormalMode.mp4')

# Abrir el video
cap = cv2.VideoCapture(video_path)

# Obtener propiedades del video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video: {fps} FPS, {width}x{height}, {total_frames} frames")

# Configurar el escritor de video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_count = 0

# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1

    # Realizar la predicción en el frame
    results = model(frame)

    # Obtener el frame con las detecciones dibujadas
    annotated_frame = results[0].plot()

    # Guardar el frame procesado en el video de salida
    out.write(annotated_frame)

    # Mostrar información cada 30 frames para no saturar la salida
    if frame_count % 30 == 0:
        print(f"Frame {frame_count}/{total_frames}")

        # Mostrar el frame (opcional, puede ser lento)
        frame_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
        cv2_imshow(frame_rgb)

        # Información de detecciones
        for r in results:
            print(f"  Objetos detectados: {len(r.boxes)}")
            for box in r.boxes:
                print(f"    Clase: {model.names[int(box.cls)]}, Confianza: {box.conf.item():.2f}")

# Liberar recursos
cap.release()
out.release()

print(f"\n✅ Video procesado guardado en: {output_video_path}")

# Mostrar el video resultante en Colab
from IPython.display import HTML
from base64 import b64encode

mp4 = open(output_video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=800 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")

Output hidden; open in https://colab.research.google.com to view.

In [4]:
from ultralytics import YOLO
import cv2
import os
from google.colab.patches import cv2_imshow
from IPython.display import display, HTML
import numpy as np

# Cargar el modelo entrenado
model = YOLO('/content/drive/MyDrive/VisionIIA/runs/segment/yolov11n_seg_custom_transfer/weights/best.pt')

# Ruta del video de entrada
video_path = '/content/drive/MyDrive/VisionIIA/banana.mp4'  # Ajusta esta ruta a tu video

# Crear directorio para guardar resultados
results_dir = 'validation_results_video'
os.makedirs(results_dir, exist_ok=True)

# Ruta del video de salida
output_video_path = os.path.join(results_dir, 'output_video_banana_NormalMode.mp4')

# Abrir el video
cap = cv2.VideoCapture(video_path)

# Obtener propiedades del video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video: {fps} FPS, {width}x{height}, {total_frames} frames")

# Configurar el escritor de video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_count = 0

# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1

    # Realizar la predicción en el frame
    results = model(frame)

    # Obtener el frame con las detecciones dibujadas
    annotated_frame = results[0].plot()

    # Guardar el frame procesado en el video de salida
    out.write(annotated_frame)

    # Mostrar información cada 30 frames para no saturar la salida
    if frame_count % 30 == 0:
        print(f"Frame {frame_count}/{total_frames}")

        # Mostrar el frame (opcional, puede ser lento)
        frame_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
        cv2_imshow(frame_rgb)

        # Información de detecciones
        for r in results:
            print(f"  Objetos detectados: {len(r.boxes)}")
            for box in r.boxes:
                print(f"    Clase: {model.names[int(box.cls)]}, Confianza: {box.conf.item():.2f}")

# Liberar recursos
cap.release()
out.release()

print(f"\n✅ Video procesado guardado en: {output_video_path}")

# Mostrar el video resultante en Colab
from IPython.display import HTML
from base64 import b64encode

mp4 = open(output_video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=800 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")

Output hidden; open in https://colab.research.google.com to view.